In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
# import logging
# import warnings

# logging.basicConfig(level=logging.DEBUG)
# warnings.filterwarnings('ignore')

In [4]:
# import tensorflow as tf

# tf.compat.v1.enable_eager_execution()

In [5]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import glowtts
import tensorflow as tf
import numpy as np
import math






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
config = malaya_speech.config.fastspeech2_config
config['encoder_hidden_size'] = 192
config['encoder_num_hidden_layers'] = 6
config['encoder_attention_head_size'] = 32
config['encoder_intermediate_size'] = 768
config = glowtts.Config(vocab_size = 66, **config)
config_glowtts = glowtts.Config_GlowTTS(malaya_speech.config.glowtts_config)

In [7]:
i = tf.placeholder(tf.int32, [None, None])
i_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

In [8]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)
    
data.keys()

dict_keys(['mel', 'text_ids', 'len_mel', 'len_text_ids', 'stop_token_target', 'f0', 'len_f0', 'energy', 'len_energy', 'g'])

In [9]:
data['text_ids'][0].shape

(72,)

In [10]:
# i = tf.convert_to_tensor([data['text_ids'][0],data['text_ids'][0]])
# i_lengths = tf.convert_to_tensor([data['len_text_ids'][0,0], data['len_text_ids'][0,0]])
# mel_outputs = tf.convert_to_tensor([data['mel'].astype(np.float32)[0],data['mel'].astype(np.float32)[0]])
# mel_lengths = tf.convert_to_tensor([408,408])

In [11]:
model = glowtts.Model(config, config_glowtts)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




In [12]:
model(i, y = mel_outputs, y_lengths = mel_lengths, training = True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


((<tf.Tensor 'glowtts/FlowSpecDecoder/mul_4:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/MatMul_2:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/MatMul_3:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/FlowSpecDecoder/add_35:0' shape=<unknown> dtype=float32>,
  <tf.Tensor 'glowtts/transpose_1:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts/Encoder/mul:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/zeros_like:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/ExpandDims:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts/ExpandDims_5:0' shape=(?, 1, ?, ?) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/ExpandDims_1:0' shape=(?, ?, 1) dtype=float32>,
  <tf.Tensor 'glowtts/mul_8:0' shape=(?, ?, 1) dtype=float32>))

In [13]:
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_) = model(i, y = mel_outputs, y_lengths = mel_lengths, training = True)
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_)

((<tf.Tensor 'glowtts_1/FlowSpecDecoder/mul_4:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts_1/MatMul_2:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts_1/MatMul_3:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts_1/FlowSpecDecoder/add_35:0' shape=<unknown> dtype=float32>,
  <tf.Tensor 'glowtts_1/transpose_1:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts_1/Encoder/mul:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts_1/Encoder/zeros_like:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts_1/Encoder/ExpandDims:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts_1/ExpandDims_5:0' shape=(?, 1, ?, ?) dtype=float32>,
  <tf.Tensor 'glowtts_1/Encoder/ExpandDims_1:0' shape=(?, ?, 1) dtype=float32>,
  <tf.Tensor 'glowtts_1/mul_8:0' shape=(?, ?, 1) dtype=float32>))

In [14]:
# commons.mle_loss(z, z_m, z_logs, logdet, z_mask)

def mle_loss(z, m, logs, logdet, mask):
    l = tf.reduce_sum(logs) + 0.5 * tf.reduce_sum(tf.math.exp(-2 * logs) * ((z - m)**2))
    l = l - tf.reduce_sum(logdet)
    l = l / tf.reduce_sum(tf.ones_like(z) * mask)
    l = l + 0.5 * math.log(2 * math.pi)
    return l

l_mle = mle_loss(z, z_m, z_logs, logdet, z_mask)
l_mle

<tf.Tensor 'add_1:0' shape=() dtype=float32>

In [15]:
# commons.duration_loss(logw, logw_, x_lengths)

def duration_loss(logw, logw_, lengths):
    # l = torch.sum((logw - logw_)**2) / torch.sum(lengths)
    l = tf.reduce_sum((logw - logw_)**2) / tf.reduce_sum(tf.cast(lengths, tf.float32))
    return l

l_length = duration_loss(logw, logw_, i_lengths)
l_length

<tf.Tensor 'truediv_1:0' shape=() dtype=float32>

In [16]:
loss_g = l_mle + l_length

In [17]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 1e-4).minimize(loss_g)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [19]:
o_ = sess.run([l_mle, l_length], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

[1.9556974, 3.9095027]

In [20]:
o_ = sess.run([attn, logw, logw_, x_mask], feed_dict = {i: data['text_ids'],
                                   i_lengths: data['len_text_ids'][:,0],
                        mel_outputs: data['mel'],
                        mel_lengths: data['len_mel'][:,0]})
o_

[array([[[[1., 1., 1., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float32),
 array([[[-1.0855805 ],
         [-2.1624827 ],
         [-2.2424912 ],
         [ 2.9815078 ],
         [ 2.369071  ],
         [-0.0254643 ],
         [ 0.9144008 ],
         [ 2.726525  ],
         [ 1.1177864 ],
         [-0.8327439 ],
         [-2.7358584 ],
         [-0.62314224],
         [-2.6793656 ],
         [-1.4983382 ],
         [ 2.022946  ],
         [-1.7799454 ],
         [ 2.964044  ],
         [-3.0438938 ],
         [ 1.2168528 ],
         [-1.6940856 ],
         [-1.2651424 ],
         [-2.5859165 ],
         [ 0.65048265],
         [ 1.2858726 ],
         [-0.23371303],
         [-1.1799268 ],
         [ 0.33944398],
         [-0.6694351 ],
         [ 0.10020971],
         [ 1.6052235 ],


In [ ]:
for k in range(300):
    print(k, sess.run([optimizer, l_mle, l_length], feed_dict = {i: data['text_ids'],
                                       i_lengths: data['len_text_ids'][:,0],
                            mel_outputs: data['mel'],
                            mel_lengths: data['len_mel'][:,0]}))

In [ ]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'test/model.ckpt')

In [ ]:
# !ls -lh test
# !rm -rf test